##  TensorFlow 循环神经网络与长短时记忆网络结构

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 8.1 循环神经网络（recurrent neural network, RNN）简介

- **传统的机器学习算法**非常依赖于人工提取的特征，使得基于传统机器学习的图像识别、语音识别以及自然语言处理等问题存在特征提取的瓶颈；
- **基于全连接神经网络**的方法也存在参数太多、无法利用数据中时间序列信息等问题。

循环神经网络挖掘数据中的时序信息以及语义信息的深度表达能力被充分利用，并在**语音识别、语言模型、机器翻译、时序分析**等方面实现了突破。

RNN的主要用途是处理和预测序列数据。RNN的来源就是为了刻画一个序列当前的输出与之前信息的关系。RNN的隐藏层之间的结点是有连接的，隐藏层的输入不仅包括输入层的输出，还包括上一时刻隐藏层的输出。RNN理论上可以被看作是同一神经网络结构被无限复制的结果。正如**卷积神经网络在不同的空间位置共享参数，循环神经网络是在不同时间位置共享参数**，从而能够使用有限的参数处理任意长度的序列。

RNN对长度为N的序列展开之后，可以视为一个有N个中间层的前馈神经网络。这个前馈神经网路没有循环连接，因此可以直接使用反向传播算法进行训练，而不需要任何特别的优化算法。这样的训练方法称为“**沿时间反向传播**” （Back-Propagation Through Time），是训练循环神经网络最常见的方法。

RNN中的状态是通过一个向量来表示的，这个向量的维度也称为RNN隐藏层的大小。假设输入向量的维度为 x ，隐藏状态的维度为 n ，那么图8.4中循环体的全连接层神经网络的输入大小为 n+x 。也就是将上一时刻的状态与当前时刻的输入拼接成一个大的向量作为循环体中神经网络的输入。因为该全连接层的输出为当前时刻的状态，于是输出层的节点个数也为 n ，循环体中的参数个数为 (n+x)×n+n 个。**循环体状态与最终输出的维度通常不同，因此为了将当前时刻的状态转化为最终的输出，RNN还需要另外一个全连接神经网络来完成这个过程。这和CNN中最后的全连接层的意义是一样的。**

在得到RNN的前向传播结果之后，可以和其他神经网络类似地定义损失函数。RNN唯一的区别在于因为它每个时刻都有一个输出，所以**RNN的总损失为所有时刻（或者部分时刻）上的损失函数的总和**。

In [2]:
# 简单的循环神经网络前向传播过程
import numpy as np

X = [1,2]
state = [0.0, 0.0]
# 分开定义不同输入部分的权重以方便操作
w_cell_state = np.asarray([[0.1, 0.2], [0.3, 0.4]])
w_cell_input = np.asarray([0.5, 0.6])
b_cell = np.asarray([0.1, -0.1])

# 定义用于输出的全连接层参数
w_output = np.asarray([[1.0], [2.0]])
b_output = 0.1

# 按照时间顺序执行RNN的前向传播过程
for i in range(len(X)):
    # 计算循环体中的全连接层神经网络
    before_activation = np.dot(state, w_cell_state) + X[i] * w_cell_input + b_cell
    state = np.tanh(before_activation)
    final_output = np.dot(state, w_output) + b_output
    print("before activation: ", before_activation)
    print("state: ", state)
    print("output: ", final_output)

before activation:  [0.6 0.5]
state:  [0.53704957 0.46211716]
output:  [1.56128388]
before activation:  [1.2923401  1.39225678]
state:  [0.85973818 0.88366641]
output:  [2.72707101]


和其他神经网络类似，在定义完损失函数之后，套用优化框架TensorFlow就可以自动完成模型训练的过程。  
理论上RNN可以支持任意长度的序列，然而在实际训练过程中，如果序列过长:
- 一方面会导致优化时出现**梯度消散和梯度爆炸**的问题
- 另一方面，展开后的前馈神经网络会**占用过大的内存**

所以实际中一般会规定一个最大长度，当序列长度超过规定长度之后会对序列进行截断。

### 8.2 长短时记忆网络（long short-term memory, LSTM）结构

RNN可以更好地利用传统神经网络结构所不能建模的信息，但也带来了更大的技术挑战一一**长期依赖（long-term dependencies）问题**。长短时记忆网络（long short-term memory，LSTM）的设计就是为了解决这个问题。与单一tanh循环体结构不同，LSTM是一种拥有三个“门”结构的特殊网络结构。

**"门结构"：LSTM 靠一些“门”的结构让信息有选择性地影响循环神经网络中每个时刻的状态。所谓“门”的结构就是一个使用sigmoid神经网络和一个按位做乘法的操作，这两个操作合在一起就是一个“门”的结构**。之所以该结构叫做“门”是因为使用sigmoid作为激活函数的全连接神经网络层会输出一个0到1之间的数值，描述当前输入有多少信息量可以通过这个结构。于是这个结构的功能就类似于一扇门， 当门打开时（sigmoid神经网络层输出为1时），全部信息都可以通过；当门关上时（sigmoid神经网络层输出为0时），任何信息都无法通过。

为了使循环神经网更有效的保存长期记忆，“遗忘门＂和“输入门”至关重要，它们是LSTM结构的核心：
- “遗忘门”的作用是让循环神经网络“忘记”之前没有用的信息。“遗忘门”会根据当前的输入和上一时刻输出决定哪一部分记忆需要被遗忘。
- 在RNN“忘记”了部分之前的状态后，它还需要从当前的输入补充最新的记忆，这个过程就是“输入门”完成的。


In [ ]:
# 定义一个LSTM结构。在TensorFlow中通过一句简单的命令就可以实现一个完整LSTM结构。
# LSTM中使用的变量也会在该函数中自动被声明。
lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm hidden size)

# 将LSTM中的状态初始化为全0数组。BasicLSTMCell类提供了zero_state函数来生成
# 全零的初始状态。state是一个包含两个张量的LSTMStateTuple类，其中state.c和
# state.h分别对应了图8.7中的c状态和h状态。
# 和其他神经网络类似，在优化循环神经网络时，每次也会使用一个batch的训练样本。
# 以下代码中，batch_size给出了一个batch的大小。
state = lstm.zero_state(batch_size, tf.float32)

# 定义损失函数
loss = 0.0
# 虽然在测试时RNN可以处理任意长度的序列，但是在训练中为了将循环网络展开成
# 前馈神经网络，我们需要知道训练数据的序列长度。在以下代码中，用num_steps来表示
# 这个长度。第9章中将介绍使用dynamic_rnn动态处理变长序列的方法。
for i in range(num_steps):
    # 在第一个时刻声明LSTM结构中使用的变量，在之后的时刻都需要复用之前定义好的变量。
    if i > 0: tf.get_variable_scope().reuse_variables()
        
    # 每一步处理时间序列中的一个时刻。将当前输入current_input（图8.7的xt)
    # 和前一时刻状态state(ht-l和ct-1）传入定义的LSTM结构可以得到当前LSTM
    # 的输出lstm_output (ht）和更新后状态state (ht和ct）。lstm_output用于输出给
    # 其他层，state用于输出给下一时刻，它们在dropout等方面可以有不同的处理方式。
    lstm_output, state = lstm(current_input, state)
    # 将当前时刻LSTM结构的输出传入一个全连接层得到最后的输出。
    final_output = fully_connected(lstm_output)
    # 计算当前时刻输出的损失。
    loss += calc_loss(final_output, expected_output)
    
# 使用类似第4章中介绍的方法训练模型。